# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), October 5, 2022**

In [1]:
using DataFrames

In [2]:
using BenchmarkTools

In [3]:
using CategoricalArrays

In [4]:
using PooledArrays

## Performance tips

### Access by column number is faster than by name

In [5]:
x = DataFrame(rand(5, 1000), :auto)
@btime $x[!, 500];
@btime $x.x500;

  2.300 ns (0 allocations: 0 bytes)
  9.600 ns (0 allocations: 0 bytes)


### When working with data `DataFrame` use barrier functions or type annotation

In [6]:
using Random
function f_bad() # this function will be slow
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    y, z = x[!, 1], x[!, 2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_bad();
# if you run @code_warntype f_bad() then you notice
# that Julia does not know column types of `DataFrame`


  96.382 ms (5999012 allocations: 122.06 MiB)


In [7]:
# solution 1 is to use barrier function (it should be possible to use it in almost any code)
function f_inner(y,z)
   p = 0.0
   for i in 1:length(y)
       p += y[i]*z[i]
   end
   p
end

function f_barrier() # extract the work to an inner function
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    f_inner(x[!, 1], x[!, 2])
end

using LinearAlgebra
function f_inbuilt() # or use inbuilt function if possible
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    dot(x[!, 1], x[!, 2])
end

@btime f_barrier();
@btime f_inbuilt();

  6.678 ms (34 allocations: 30.52 MiB)
  6.414 ms (34 allocations: 30.52 MiB)


In [8]:
# solution 2 is to provide the types of extracted columns
# it is simpler but there are cases in which you will not know these types
# This example  assumes that you have DataFrames master at least from August 31, 2018
function f_typed()
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    y::Vector{Float64}, z::Vector{Float64} = x[!, 1], x[!, 2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_typed();

  7.002 ms (34 allocations: 30.52 MiB)


In general for tall and narrow tables it is often useful to use `Tables.rowtable`, `Tables.columntable` or `Tables.namedtupleiterator` for intermediate processing of data in a type-stable way.

### Consider using delayed `DataFrame` creation technique

also notice the difference in performance between copying vs non-copying data frame creation

In [9]:
function f1()
    x = DataFrame([Vector{Float64}(undef, 10^4) for i in 1:100], :auto, copycols=false) # we work with a DataFrame directly
    for c in 1:ncol(x)
        d = x[!, c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f1a()
    x = DataFrame([Vector{Float64}(undef, 10^4) for i in 1:100], :auto) # we work with a DataFrame directly
    for c in 1:ncol(x)
        d = x[!, c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f2()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame(x, :auto, copycols=false) # we delay creation of DataFrame after we have our job done
end

function f2a()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame(x, :auto) # we delay creation of DataFrame after we have our job done
end

@btime f1();
@btime f1a();
@btime f2();
@btime f2a();

  26.124 ms (1949524 allocations: 37.40 MiB)
  28.348 ms (1949724 allocations: 45.03 MiB)
  1.282 ms (624 allocations: 7.66 MiB)
  2.226 ms (824 allocations: 15.30 MiB)


### You can add rows to a `DataFrame` in place and it is fast

In [10]:
x = DataFrame(rand(10^6, 5), :auto)
y = DataFrame(transpose(1.0:5.0), :auto)
z = [1.0:5.0;]

@btime vcat($x, $y); # creates a new DataFrame - slow
@btime append!($x, $y); # in place - fast

x = DataFrame(rand(10^6, 5), :auto) # reset to the same starting point
@btime push!($x, $z); # add a single row in place - fast

  5.930 ms (191 allocations: 38.16 MiB)
  815.584 ns (17 allocations: 672 bytes)
  432.161 ns (16 allocations: 256 bytes)


### Allowing `missing` as well as `categorical` slows down computations

In [11]:
using StatsBase

function test(data) # uses countmap function to test performance
    println(eltype(data))
    x = rand(data, 10^6)
    y = categorical(x)
    println(" raw:")
    @btime countmap($x)
    println(" categorical:")
    @btime countmap($y)
    nothing
end

test(1:10)
test([randstring() for i in 1:10])
test(allowmissing(1:10))
test(allowmissing([randstring() for i in 1:10]))


Int64
 raw:
  3.360 ms (7 allocations: 7.63 MiB)
 categorical:
  15.130 ms (4 allocations: 688 bytes)
String
 raw:
  32.054 ms (4 allocations: 528 bytes)
 categorical:
  29.256 ms (4 allocations: 688 bytes)
Union{Missing, Int64}
 raw:
  5.275 ms (4 allocations: 560 bytes)
 categorical:
  15.310 ms (1000004 allocations: 30.52 MiB)
Union{Missing, String}
 raw:
  20.235 ms (4 allocations: 528 bytes)
 categorical:
  26.919 ms (1000004 allocations: 30.52 MiB)


### When aggregating use column selector and prefer integer, categorical, or pooled array grouping variable

In [12]:
df = DataFrame(x=rand('a':'d', 10^7), y=1);

In [13]:
gdf = groupby(df, :x)

Row,x,y
,Char,Int64
1,c,1
2,c,1
3,c,1
4,c,1
5,c,1
6,c,1
7,c,1
8,c,1
9,c,1


In [14]:
@btime combine(v -> sum(v.y), $gdf) # traditional syntax, slow

  39.068 ms (319 allocations: 19.10 MiB)


Row,x,x1
,Char,Int64
1,c,2501452
2,a,2502167
3,d,2500969
4,b,2495412


In [15]:
@btime combine($gdf, :y=>sum) # use column selector

  8.999 ms (226 allocations: 14.12 KiB)


Row,x,y_sum
,Char,Int64
1,c,2501452
2,a,2502167
3,d,2500969
4,b,2495412


In [16]:
transform!(df, :x => categorical => :x);

In [17]:
gdf = groupby(df, :x)

Row,x,y
,Cat…,Int64
1,a,1
2,a,1
3,a,1
4,a,1
5,a,1
6,a,1
7,a,1
8,a,1
9,a,1


In [18]:
@btime combine($gdf, :y=>sum)

  9.055 ms (233 allocations: 14.70 KiB)


Row,x,y_sum
,Cat…,Int64
1,a,2502167
2,b,2495412
3,c,2501452
4,d,2500969


In [19]:
transform!(df, :x => PooledArray{Char} => :x)

Row,x,y
,Char,Int64
1,c,1
2,a,1
3,c,1
4,d,1
5,d,1
6,b,1
7,d,1
8,c,1
9,c,1


In [20]:
gdf = groupby(df, :x)

Row,x,y
,Char,Int64
1,c,1
2,c,1
3,c,1
4,c,1
5,c,1
6,c,1
7,c,1
8,c,1
9,c,1


In [21]:
@btime combine($gdf, :y=>sum)

  8.790 ms (228 allocations: 14.19 KiB)


Row,x,y_sum
,Char,Int64
1,c,2501452
2,a,2502167
3,d,2500969
4,b,2495412


### Use views instead of materializing a new DataFrame

In [22]:
x = DataFrame(rand(100, 1000), :auto)

Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,0.0903367,0.163747,0.684125,0.359433,0.391201,0.553945,0.386801,0.170243,0.785965,0.37364,0.121847,0.319183,0.318759,0.816578,0.158892,0.146362,0.0662521,0.293301,0.965506,0.992222,0.387689,0.901977,0.266571,0.904297,0.996896,0.822812,0.00759657,0.0638031,0.231909,0.199771,0.971534,0.795822,0.258391,0.99457,0.718742,0.85089,0.836376,0.264602,0.208107,0.046411,0.289224,0.329341,0.414283,0.597822,0.9839,0.359793,0.676169,0.595102,0.85256,0.411566,0.848235,0.114632,0.280953,0.146555,0.113455,0.282086,0.703041,0.240051,0.490823,0.539732,0.623681,0.988882,0.457174,0.465744,0.182501,0.465968,0.961894,0.680664,0.70148,0.480709,0.338984,0.0535537,0.794744,0.403971,0.481755,0.541795,0.915109,0.245369,0.172523,0.677059,0.152196,0.487205,0.110634,0.715605,0.781465,0.833483,0.244138,0.789438,0.407008,0.476403,0.677984,0.173702,0.461884,0.730055,0.489883,0.583402,0.272328,0.403047,0.946289,0.174344,⋯
2,0.344177,0.69609,0.486624,0.493705,0.582797,0.173189,0.820015,0.0771206,0.887312,0.96985,0.221445,0.0222909,0.35504,0.151972,0.162617,0.367808,0.0210497,0.835547,0.273021,0.629958,0.371279,0.271105,0.810216,0.994883,0.859452,0.604366,0.242305,0.179147,0.100762,0.426716,0.387399,0.788839,0.528445,0.219166,0.785292,0.0635088,0.986095,0.524804,0.565857,0.650462,0.618702,0.522669,0.330604,0.315848,0.602057,0.74021,0.689685,0.613858,0.658094,0.63099,0.0457496,0.1218,0.662505,0.486686,0.965805,0.735053,0.317709,0.101198,0.69567,0.92604,0.985665,0.97178,0.172371,0.371873,0.523836,0.835936,0.132122,0.580123,0.158737,0.0112453,0.678056,0.963532,0.852347,0.809769,0.0955158,0.139168,0.895515,0.123145,0.920123,0.0959729,0.166062,0.972508,0.529295,0.681649,0.34539,0.380668,0.31861,0.962136,0.500649,0.807215,0.142792,0.591934,0.592058,0.87848,0.457947,0.430468,0.23756,0.751663,0.0676139,0.899255,⋯
3,0.521849,0.253648,0.40574,0.306684,0.966418,0.828701,0.393079,0.4522,0.558392,0.318071,0.247652,0.789553,0.188888,0.420516,0.0553191,0.0735833,0.931054,0.258493,0.565888,0.347106,0.670604,0.0613075,0.676572,0.00357315,0.348651,0.355546,0.770769,0.772471,0.952631,0.675666,0.326898,0.325276,0.836149,0.547165,0.158519,0.075259,0.74239,0.311487,0.747634,0.5441,0.821079,0.116565,0.330566,0.842012,0.156957,0.846349,0.375048,0.530688,0.578145,0.598006,0.911034,0.49991,0.700347,0.0469332,0.47465,0.671266,0.561399,0.569183,0.309344,0.109321,0.441051,0.34027,0.519565,0.457979,0.0413752,0.622941,0.472425,0.275293,0.767976,0.473561,0.862745,0.892284,0.170098,0.0492216,0.630674,0.438646,0.99443,0.608574,0.395431,0.743631,0.4453,0.618449,0.38405,0.706232,0.84794,0.282177,0.320543,0.119825,0.186886,0.210828,0.845314,0.861254,0.491505,0.682332,0.105684,0.634092,0.205769,0.764452,0.224076,0.148745,⋯
4,0.0239116,0.505325,6.37694e-5,0.347586,0.330535,0.901262,0.37116,0.627739,0.593704,0.294418,0.980

In [23]:
@btime $x[1:1, :]

  158.100 μs (2986 allocations: 159.50 KiB)


Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,0.0903367,0.163747,0.684125,0.359433,0.391201,0.553945,0.386801,0.170243,0.785965,0.37364,0.121847,0.319183,0.318759,0.816578,0.158892,0.146362,0.0662521,0.293301,0.965506,0.992222,0.387689,0.901977,0.266571,0.904297,0.996896,0.822812,0.00759657,0.0638031,0.231909,0.199771,0.971534,0.795822,0.258391,0.99457,0.718742,0.85089,0.836376,0.264602,0.208107,0.046411,0.289224,0.329341,0.414283,0.597822,0.9839,0.359793,0.676169,0.595102,0.85256,0.411566,0.848235,0.114632,0.280953,0.146555,0.113455,0.282086,0.703041,0.240051,0.490823,0.539732,0.623681,0.988882,0.457174,0.465744,0.182501,0.465968,0.961894,0.680664,0.70148,0.480709,0.338984,0.0535537,0.794744,0.403971,0.481755,0.541795,0.915109,0.245369,0.172523,0.677059,0.152196,0.487205,0.110634,0.715605,0.781465,0.833483,0.244138,0.789438,0.407008,0.476403,0.677984,0.173702,0.461884,0.730055,0.489883,0.583402,0.272328,0.403047,0.946289,0.174344,⋯


In [24]:
@btime $x[1, :]

  23.618 ns (0 allocations: 0 bytes)


Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,0.0903367,0.163747,0.684125,0.359433,0.391201,0.553945,0.386801,0.170243,0.785965,0.37364,0.121847,0.319183,0.318759,0.816578,0.158892,0.146362,0.0662521,0.293301,0.965506,0.992222,0.387689,0.901977,0.266571,0.904297,0.996896,0.822812,0.00759657,0.0638031,0.231909,0.199771,0.971534,0.795822,0.258391,0.99457,0.718742,0.85089,0.836376,0.264602,0.208107,0.046411,0.289224,0.329341,0.414283,0.597822,0.9839,0.359793,0.676169,0.595102,0.85256,0.411566,0.848235,0.114632,0.280953,0.146555,0.113455,0.282086,0.703041,0.240051,0.490823,0.539732,0.623681,0.988882,0.457174,0.465744,0.182501,0.465968,0.961894,0.680664,0.70148,0.480709,0.338984,0.0535537,0.794744,0.403971,0.481755,0.541795,0.915109,0.245369,0.172523,0.677059,0.152196,0.487205,0.110634,0.715605,0.781465,0.833483,0.244138,0.789438,0.407008,0.476403,0.677984,0.173702,0.461884,0.730055,0.489883,0.583402,0.272328,0.403047,0.946289,0.174344,⋯


In [25]:
@btime view($x, 1:1, :)

  24.422 ns (0 allocations: 0 bytes)


Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,0.0903367,0.163747,0.684125,0.359433,0.391201,0.553945,0.386801,0.170243,0.785965,0.37364,0.121847,0.319183,0.318759,0.816578,0.158892,0.146362,0.0662521,0.293301,0.965506,0.992222,0.387689,0.901977,0.266571,0.904297,0.996896,0.822812,0.00759657,0.0638031,0.231909,0.199771,0.971534,0.795822,0.258391,0.99457,0.718742,0.85089,0.836376,0.264602,0.208107,0.046411,0.289224,0.329341,0.414283,0.597822,0.9839,0.359793,0.676169,0.595102,0.85256,0.411566,0.848235,0.114632,0.280953,0.146555,0.113455,0.282086,0.703041,0.240051,0.490823,0.539732,0.623681,0.988882,0.457174,0.465744,0.182501,0.465968,0.961894,0.680664,0.70148,0.480709,0.338984,0.0535537,0.794744,0.403971,0.481755,0.541795,0.915109,0.245369,0.172523,0.677059,0.152196,0.487205,0.110634,0.715605,0.781465,0.833483,0.244138,0.789438,0.407008,0.476403,0.677984,0.173702,0.461884,0.730055,0.489883,0.583402,0.272328,0.403047,0.946289,0.174344,⋯


In [26]:
@btime $x[1:1, 1:20]

  3.750 μs (51 allocations: 4.28 KiB)


Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0903367,0.163747,0.684125,0.359433,0.391201,0.553945,0.386801,0.170243,0.785965,0.37364,0.121847,0.319183,0.318759,0.816578,0.158892,0.146362,0.0662521,0.293301,0.965506,0.992222


In [27]:
@btime $x[1, 1:20]

  29.548 ns (0 allocations: 0 bytes)


Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0903367,0.163747,0.684125,0.359433,0.391201,0.553945,0.386801,0.170243,0.785965,0.37364,0.121847,0.319183,0.318759,0.816578,0.158892,0.146362,0.0662521,0.293301,0.965506,0.992222


In [28]:
@btime view($x, 1:1, 1:20)

  27.739 ns (0 allocations: 0 bytes)


Row,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0903367,0.163747,0.684125,0.359433,0.391201,0.553945,0.386801,0.170243,0.785965,0.37364,0.121847,0.319183,0.318759,0.816578,0.158892,0.146362,0.0662521,0.293301,0.965506,0.992222
